In [25]:
import pandas as pd
import glob
import os

folder_path = './matches/'
all_files = glob.glob(os.path.join(folder_path, "*.xlsx"))

df_list = []
for file in all_files:
    try:
        temp_df = pd.read_excel(file)
        df_list.append(temp_df)
    except Exception as e:
        print(f"Ошибка при чтении {file}: {e}")

df_all = pd.concat(df_list, ignore_index=True)

columns_needed = ['Kill', 'Death', 'Assist', '1K', '2K', '3K', '4K', '5K',
                  '1v1', '1v2', '1v3', '1v4', '1v5', 'FK', 'Rounds']
df_all[columns_needed] = df_all[columns_needed].apply(pd.to_numeric, errors='coerce')

df_grouped = df_all.groupby(['Team', 'Player']).agg({
    'Kill': 'sum',
    'Death': 'sum',
    'Assist': 'sum',
    '1K': 'sum',
    '2K': 'sum',
    '3K': 'sum',
    '4K': 'sum',
    '5K': 'sum',
    '1v1': 'sum',
    '1v2': 'sum',
    '1v3': 'sum',
    '1v4': 'sum',
    '1v5': 'sum',
    'FK': 'sum',
    'Rounds': 'sum'
}).reset_index()

df_grouped['KPR'] = df_grouped['Kill'] / df_grouped['Rounds']
df_grouped['Surviving Rate'] = (df_grouped['Rounds'] - df_grouped['Death']) / df_grouped['Rounds']
df_grouped['Impact'] = (
    df_grouped['1K'] +
    df_grouped['2K'] * 2 +
    df_grouped['3K'] * 3 +
    df_grouped['4K'] * 4 +
    df_grouped['5K'] * 5
) / df_grouped['Rounds']
df_grouped['Assist Rate'] = df_grouped['Assist'] / df_grouped['Rounds']
df_grouped['FK Rate'] = df_grouped['FK'] / df_grouped['Rounds']

normalization = {
    'KPR': 0.70,
    'Surviving Rate': 0.63,
    'Impact': 1.10,
    'Assist Rate': 0.10,
    'FK Rate': 0.13
}

df_grouped['KPR'] /= normalization['KPR']
df_grouped['Surviving Rate'] /= normalization['Surviving Rate']
df_grouped['Impact'] /= normalization['Impact']
df_grouped['Assist Rate'] /= normalization['Assist Rate']
df_grouped['FK Rate'] /= normalization['FK Rate']

weights = {
    'KPR': 0.40,
    'Surviving Rate': 0.25,
    'Impact': 0.15,
    'Assist Rate': 0.15,
    'FK Rate': 0.05
}

df_grouped['Rating'] = (
    df_grouped['KPR'] * weights['KPR'] +
    df_grouped['Surviving Rate'] * weights['Surviving Rate'] +
    df_grouped['Impact'] * weights['Impact'] +
    df_grouped['Assist Rate'] * weights['Assist Rate'] +
    df_grouped['FK Rate'] * weights['FK Rate']
)

metrics_to_round = [
    'KPR', 'Surviving Rate', 'Impact',
    'Assist Rate', 'FK Rate', 'Rating'
]

df_grouped[metrics_to_round] = df_grouped[metrics_to_round].round(2)
df_grouped = df_grouped.sort_values(by=['Team', 'Rating'], ascending=[True, False])
df_grouped.to_excel('cs2_rating.xlsx', index=False)